In [1]:
import os
os.chdir("/Users/DanPfeiffer/Documents/code/data-collector/src")

In [86]:
import pandas as pd
from geo import geocode, timezones
import importlib
import plotly.express as px
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
importlib.reload(geocode)
importlib.reload(timezones)
importlib.reload(px)

<module 'plotly.express' from '/Users/DanPfeiffer/Documents/code/data-collector/env/lib/python3.11/site-packages/plotly/express/__init__.py'>

In [3]:
df = pd.read_csv("../output/box_scores/box_scores_df.csv")

In [26]:
#box_report = ProfileReport(box_scores_df,
#                           explorative=True)

In [27]:
# box_report.to_file("../output/box_scores/box_score_report.html")

In [56]:
box_scores_df = df.copy()
box_scores_df.columns

Index(['id', 'uid', 'date', 'attendance', 'time_valid', 'neutral_site',
       'conference_competition', 'play_by_play_available', 'recent',
       'start_date', 'notes_type', 'notes_headline', 'broadcast_market',
       'broadcast_name', 'type_id', 'type_abbreviation', 'venue_id',
       'venue_full_name', 'venue_address_city', 'venue_address_state',
       'venue_indoor', 'status_clock', 'status_display_clock', 'status_period',
       'status_type_id', 'status_type_name', 'status_type_state',
       'status_type_completed', 'status_type_description',
       'status_type_detail', 'status_type_short_detail', 'status_is_tbd_flex',
       'format_regulation_periods', 'home_id', 'home_uid', 'home_location',
       'home_name', 'home_abbreviation', 'home_display_name',
       'home_short_display_name', 'home_color', 'home_alternate_color',
       'home_is_active', 'home_venue_id', 'home_logo', 'home_score',
       'home_winner', 'home_current_rank', 'home_linescores', 'home_records',
     

In [5]:
cols_to_keep = ['id', 'uid', 'date', 'attendance', 'neutral_site',
       'start_date', 'notes_type', 'venue_id',
       'venue_full_name', 'venue_address_city', 'venue_address_state',
       'venue_indoor',      'home_id', 'home_uid', 'home_location',
       'home_name', 'home_abbreviation', 'home_display_name',
       'home_short_display_name',
       'home_is_active', 'home_venue_id', 'home_score',
       'home_winner',       
       'away_id', 'away_uid', 'away_location', 'away_name',
       'away_abbreviation', 'away_display_name', 'away_short_display_name',
       'away_is_active', 'away_venue_id',
       'away_logo', 'away_score', 'away_winner', 'game_id', 'season', 'season_type',
       'week', 'status_type_alt_detail']

In [6]:
box_scores_df = box_scores_df[cols_to_keep]

In [57]:
venue_id_mapper = box_scores_df[box_scores_df["venue_full_name"].notnull()][[
    "venue_id", 
    "venue_full_name", 
    "venue_address_city",
    "venue_address_state"
]].groupby([
    "venue_id", 
    "venue_full_name",
    "venue_address_city",
    "venue_address_state"
], dropna=False).mean().reset_index().sort_values(by=[
    "venue_address_state", 
    "venue_address_city"],
    na_position="first")
venue_id_mapper.rename(columns={"venue_full_name": "lookup_venue_full_name",
                                "venue_address_city": "lookup_venue_address_city",
                                "venue_address_state": "lookup_venue_address_state"}, inplace=True)
print(venue_id_mapper)

    venue_id           lookup_venue_full_name lookup_venue_address_city  \
27    3766.0             Independence Stadium                       NaN   
8     1875.0                Frankfurt Stadium                 Frankfurt   
9     2455.0                  Wembley Stadium                    London   
51    5128.0               Twickenham Stadium                    London   
54    5534.0        Tottenham Hotspur Stadium                    London   
7     1850.0                   Estadio Azteca               Mexico City   
5     1775.0                    Allianz Arena                    Munich   
47    3970.0               State Farm Stadium                  Glendale   
4      538.0       Dignity Health Sports Park                    Carson   
56    7065.0                     SoFi Stadium                 Inglewood   
3      477.0    Los Angeles Memorial Coliseum               Los Angeles   
35    3867.0                 Oakland Coliseum                   Oakland   
41    3932.0             

In [58]:
box_scores_df["venue_id"] = box_scores_df["venue_id"].combine_first(box_scores_df["home_venue_id"])

In [59]:
box_scores_df = pd.merge(box_scores_df, venue_id_mapper, how="left", on="venue_id")

In [60]:
box_scores_df["venue_full_name"] = box_scores_df["venue_full_name"].combine_first(box_scores_df["lookup_venue_full_name"])
box_scores_df["venue_address_city"] = box_scores_df["venue_address_city"].combine_first(box_scores_df["lookup_venue_address_city"])
box_scores_df["venue_address_state"] = box_scores_df["venue_address_state"].combine_first(box_scores_df["lookup_venue_address_state"])
box_scores_df.drop(columns=["lookup_venue_full_name",
                            "lookup_venue_address_city",
                            "lookup_venue_address_state"], inplace=True)

In [7]:
box_scores_df = box_scores_df.loc[box_scores_df["season_type"] != 1]

In [39]:
box_scores_df["venue_id"].fillna(box_scores_df["home_venue_id"], inplace=True)

In [61]:
stadium_df = box_scores_df[[
    "venue_id", 
    "venue_full_name", 
    "venue_address_city",
    "venue_address_state",
    "attendance"
]].groupby([
    "venue_id", 
    "venue_full_name",
    "venue_address_city",
    "venue_address_state"
], dropna=False).mean().reset_index().sort_values(by=[
    "venue_address_state", 
    "venue_address_city"],
    na_position="first")

In [62]:
stadium_df[stadium_df["venue_full_name"].isnull()]

,venue_id,venue_full_name,venue_address_city,venue_address_state,attendance
57,NaN,NaN,NaN,NaN,0.0


In [64]:
stadium_df.dropna(subset=["venue_id"], inplace=True)

In [68]:
city_st_mapper = {
    "Frankfurt": "DEU",
    "London": "ENG",
    "Mexico City": "MEX",
    "Munich": "DEU"
}
stadium_df["venue_address_state"].fillna(stadium_df["venue_address_city"].map(city_st_mapper), inplace=True)

In [70]:
stadium_df.loc[stadium_df["venue_full_name"] == "Independence Stadium", "venue_address_city"] = "Shreveport"
stadium_df.loc[stadium_df["venue_full_name"] == "Independence Stadium", "venue_address_state"] = "LA"

In [72]:
stadium_df = geocode.geocode_dataframe(df=stadium_df, 
                                       city_colname="venue_address_city", 
                                       state_colname="venue_address_state")

In [74]:
stadium_df.sort_values(by=["venue_address_state", "venue_address_city", "venue_id"])

,venue_id,venue_full_name,venue_address_city,venue_address_state,attendance,latitude,longitude
47,3970.0,State Farm Stadium,Glendale,AZ,49622.353535,33.538652,-112.185987
4,538.0,Dignity Health Sports Park,Carson,CA,21525.607143,33.831674,-118.281693
56,7065.0,SoFi Stadium,Inglewood,CA,56372.692308,33.961680,-118.353131
3,477.0,Los Angeles Memorial Coliseum,Los Angeles,CA,64275.684211,34.054908,-118.242643
35,3867.0,Oakland Coliseum,Oakland,CA,43179.993103,37.804351,-122.271164
41,3932.0,SDCCU Stadium,San Diego,CA,52944.206349,32.715738,-117.161084
1,24.0,Monster Park,San Francisco,CA,53753.789474,37.774929,-122.419415
16,3669.0,Candlestick Park,San Francisco,CA,51800.767123,37.774929,-122.419415
50,4738.0,Levi's Stadium,Santa Clara,CA,58688.404040,37.354108,-121.955236
2,50.0,Invesco Field,Denver,CO,58445.294118,39.739236,-104.990251


In [87]:
tz_series, offset_series = timezones.get_time_zone_and_offset(lat_series=stadium_df["latitude"],
                                                                                lon_series=stadium_df["longitude"])
stadium_df["timezone"] = tz_series
stadium_df["timezone_shift"] = offset_series

In [88]:
stadium_df

,venue_id,venue_full_name,venue_address_city,venue_address_state,attendance,latitude,longitude,timezone_shift,timezone
27,3766.0,Independence Stadium,Shreveport,LA,0.000000,32.525152,-93.750179,-5.0,America/Chicago
8,1875.0,Frankfurt Stadium,Frankfurt,DEU,50083.500000,50.110922,8.682127,2.0,Europe/Berlin
9,2455.0,Wembley Stadium,London,ENG,83825.720000,51.507218,-0.127586,1.0,Europe/London
51,5128.0,Twickenham Stadium,London,ENG,74031.333333,51.507218,-0.127586,1.0,Europe/London
54,5534.0,Tottenham Hotspur Stadium,London,ENG,60733.750000,51.507218,-0.127586,1.0,Europe/London
7,1850.0,Estadio Azteca,Mexico City,MEX,77127.250000,19.432608,-99.133208,-6.0,America/Mexico_City
5,1775.0,Allianz Arena,Munich,DEU,69811.000000,48.135125,11.581981,2.0,Europe/Berlin
47,3970.0,State Farm Stadium,Glendale,AZ,49622.353535,33.538652,-112.185987,-7.0,America/Phoenix
4,538.0,Dignity Health Sports Park,Carson,CA,21525.607143,33.831674,-118.281693,-7.0,America/Los_Angeles
56,7065.0,SoFi Stadium,Inglewood,CA,56372.692308,33.961680,-118.353131,-7.0,America/Los_Angeles


In [89]:
stadium_df.to_csv("reference/stadiums.csv", index=False)

In [50]:
box_scores_df["score_differential"] = box_scores_df["home_score"] - box_scores_df["away_score"]
box_scores_df["abs_score_differential"] = abs(box_scores_df["score_differential"])
box_scores_df["total_score"] = box_scores_df["home_score"] + box_scores_df["away_score"]
box_scores_df["score_differential_percentile"] = box_scores_df['score_differential'].rank(pct=True)
box_scores_df["total_score_percentile"] = box_scores_df['total_score'].rank(pct=True)

In [51]:
def calculate_percentile(group, col):
    group[f'yearly_{col}_percentile'] = group[col].rank(pct=True)
    return group

box_scores_df = box_scores_df.groupby('season').apply(calculate_percentile, "score_differential").reset_index(drop=True)
box_scores_df = box_scores_df.groupby("season").apply(calculate_percentile, "total_score").reset_index(drop=True)

In [52]:
max_reg = box_scores_df.groupby("season")["week"].max().reset_index()
max_reg = max_reg.rename(columns={"week": "max_regular_season_week"})
box_scores_df = pd.merge(box_scores_df, max_reg, on="season", how="left")

,id,uid,date,attendance,neutral_site,start_date,notes_type,venue_id,venue_full_name,venue_address_city,venue_address_state,venue_indoor,home_id,home_uid,home_location,home_name,home_abbreviation,home_display_name,home_short_display_name,home_is_active,home_venue_id,home_score,home_winner,away_id,away_uid,away_location,away_name,away_abbreviation,away_display_name,away_short_display_name,away_is_active,away_venue_id,away_logo,away_score,away_winner,game_id,season,season_type,week,status_type_alt_detail,score_differential,abs_score_differential,total_score,score_differential_percentile,total_score_percentile,yearly_score_differential_percentile,yearly_total_score_percentile,max_regular_season_week
0,250102002,s:20~l:28~e:250102002~c:250102002,2005-01-02T18:00Z,73414,False,2005-01-02T18:00Z,NaN,3883.0,Highmark Stadium,Orchard Park,NY,False,2,s:20~l:28~t:2,Buffalo,Bills,BUF,Buffalo Bills,Bills,True,3883.0,24,False,23,s:20~l:28~t:23,Pittsburgh,Steelers,PIT,Pittsburgh Steelers,Steelers,True,3752.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,29,True,250102002,2004,2,17.0,NaN,-5,5,53,0.295764,0.739409,0.285714,0.767857,17.0
1,250102003,s:20~l:28~e:250102003~c:250102003,2005-01-02T18:00Z,62197,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,3,s:20~l:28~t:3,Chicago,Bears,CHI,Chicago Bears,Bears,True,3933.0,14,False,9,s:20~l:28~t:9,Green Bay,Packers,GB,Green Bay Packers,Packers,True,3798.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,31,True,250102003,2004,2,17.0,NaN,-17,17,45,0.094637,0.534298,0.071429,0.517857,17.0
2,250102010,s:20~l:28~e:250102010~c:250102010,2005-01-02T18:00Z,68809,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,10,s:20~l:28~t:10,Tennessee,Titans,TEN,Tennessee Titans,Titans,True,3810.0,24,True,8,s:20~l:28~t:8,Detroit,Lions,DET,Detroit Lions,Lions,True,3727.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,19,False,250102010,2004,2,17.0,NaN,5,5,43,0.595317,0.464050,0.625000,0.464286,17.0
3,250102014,s:20~l:28~e:250102014~c:250102014,2005-01-02T18:00Z,65877,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,14,s:20~l:28~t:14,St. Louis,Rams,STL,St. Louis Rams,Rams,False,3494.0,32,True,20,s:20~l:28~t:20,New York,Jets,NYJ,New York Jets,Jets,True,3839.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,29,False,250102014,2004,2,17.0,OT,3,3,61,0.521279,0.868733,0.500000,0.857143,17.0
4,250102017,s:20~l:28~e:250102017~c:250102017,2005-01-02T18:00Z,68756,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,17,s:20~l:28~t:17,New England,Patriots,NE,New England Patriots,Patriots,True,3738.0,21,True,25,s:20~l:28~t:25,San Francisco,49ers,SF,San Francisco 49ers,49ers,True,3669.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,7,False,250102017,2004,2,17.0,NaN,14,14,28,0.800719,0.111640,0.821429,0.142857,17.0


In [53]:
box_scores_df["week"] = (box_scores_df["season_type"] - 2) * box_scores_df["max_regular_season_week"] + box_scores_df["week"]

In [55]:
box_scores_df.head(50)

,id,uid,date,attendance,neutral_site,start_date,notes_type,venue_id,venue_full_name,venue_address_city,venue_address_state,venue_indoor,home_id,home_uid,home_location,home_name,home_abbreviation,home_display_name,home_short_display_name,home_is_active,home_venue_id,home_score,home_winner,away_id,away_uid,away_location,away_name,away_abbreviation,away_display_name,away_short_display_name,away_is_active,away_venue_id,away_logo,away_score,away_winner,game_id,season,season_type,week,status_type_alt_detail,score_differential,abs_score_differential,total_score,score_differential_percentile,total_score_percentile,yearly_score_differential_percentile,yearly_total_score_percentile,max_regular_season_week
0,250102002,s:20~l:28~e:250102002~c:250102002,2005-01-02T18:00Z,73414,False,2005-01-02T18:00Z,NaN,3883.0,Highmark Stadium,Orchard Park,NY,False,2,s:20~l:28~t:2,Buffalo,Bills,BUF,Buffalo Bills,Bills,True,3883.0,24,False,23,s:20~l:28~t:23,Pittsburgh,Steelers,PIT,Pittsburgh Steelers,Steelers,True,3752.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,29,True,250102002,2004,2,17.0,NaN,-5,5,53,0.295764,0.739409,0.285714,0.767857,17.0
1,250102003,s:20~l:28~e:250102003~c:250102003,2005-01-02T18:00Z,62197,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,3,s:20~l:28~t:3,Chicago,Bears,CHI,Chicago Bears,Bears,True,3933.0,14,False,9,s:20~l:28~t:9,Green Bay,Packers,GB,Green Bay Packers,Packers,True,3798.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,31,True,250102003,2004,2,17.0,NaN,-17,17,45,0.094637,0.534298,0.071429,0.517857,17.0
2,250102010,s:20~l:28~e:250102010~c:250102010,2005-01-02T18:00Z,68809,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,10,s:20~l:28~t:10,Tennessee,Titans,TEN,Tennessee Titans,Titans,True,3810.0,24,True,8,s:20~l:28~t:8,Detroit,Lions,DET,Detroit Lions,Lions,True,3727.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,19,False,250102010,2004,2,17.0,NaN,5,5,43,0.595317,0.464050,0.625000,0.464286,17.0
3,250102014,s:20~l:28~e:250102014~c:250102014,2005-01-02T18:00Z,65877,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,14,s:20~l:28~t:14,St. Louis,Rams,STL,St. Louis Rams,Rams,False,3494.0,32,True,20,s:20~l:28~t:20,New York,Jets,NYJ,New York Jets,Jets,True,3839.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,29,False,250102014,2004,2,17.0,OT,3,3,61,0.521279,0.868733,0.500000,0.857143,17.0
4,250102017,s:20~l:28~e:250102017~c:250102017,2005-01-02T18:00Z,68756,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,17,s:20~l:28~t:17,New England,Patriots,NE,New England Patriots,Patriots,True,3738.0,21,True,25,s:20~l:28~t:25,San Francisco,49ers,SF,San Francisco 49ers,49ers,True,3669.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,7,False,250102017,2004,2,17.0,NaN,14,14,28,0.800719,0.111640,0.821429,0.142857,17.0
5,250102021,s:20~l:28~e:250102021~c:250102021,2005-01-02T18:00Z,67074,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,21,s:20~l:28~t:21,Philadelphia,Eagles,PHI,Philadelphia Eagles,Eagles,True,3806.0,10,False,4,s:20~l:28~t:4,Cincinnati,Bengals,CIN,Cincinnati Bengals,Bengals,True,3874.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,38,True,250102021,2004,2,17.0,NaN,-28,28,48,0.022056,0.613389,0.035714,0.660714,17.0
6,250102028,s:20~l:28~e:250102028~c:250102028,2005-01-02T18:00Z,76876,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,28,s:20~l:28~t:28,Washington,Redskins,WSH,Washington Redskins,Redskins,False,3719.0,21,True,16,s:20~l:28~t:16,Minnesota,Vikings,MIN,Minnesota Vikings,Vikings,True,16.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,18,False,250102028,2004,2,17.0,NaN,3,3,39,0.521279,0.346094,0.500000,0.339286,17.0
7,250102029,s:20~l:28~e:250102029~c:250102029,2005-01-02T18:00Z,73302,False,2005-01-02T18:00Z,NaN,NaN,NaN,NaN,NaN,NaN,29,s:20~l:28~t:29,Carolina,Panthers,CAR,Carolina Panthers,Panthers,True,3628.0,18,False,18,s:20~l:28~t:18,New Orleans,Saints,NO,New Orleans Saints,Saints,True,3493.0,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,21,True,250102029,2004,2,17.0,NaN,-3,3,39,0.363972,0.346094,0.357

In [56]:
score_cols = ["home_score",
    "away_score",
    "score_differential",
    "abs_score_differential",
    "total_score"]
box_scores_df[score_cols].describe()

,home_score,away_score,score_differential,abs_score_differential,total_score
count,5146.000000,5146.000000,5146.000000,5146.000000,5146.000000
mean,23.508550,21.309949,2.198601,11.570152,44.818500
std,10.310855,10.025765,14.653477,9.255526,14.104431
min,0.000000,0.000000,-49.000000,0.000000,0.000000
25%,17.000000,14.000000,-7.000000,4.000000,35.000000
50%,23.000000,21.000000,3.000000,8.000000,44.000000
75%,30.000000,28.000000,11.000000,17.000000,54.000000
max,70.000000,59.000000,59.000000,59.000000,105.000000


In [57]:
fig = px.histogram(box_scores_df, x=["home_score", "away_score"], histnorm="percent")
fig.write_html("../output/visualizations/histogram/score_histogram.html")

In [63]:
for col in score_cols:
    title = f"{col}_histogram"
    fig = px.histogram(box_scores_df, x=col, title=title, histnorm="percent")
    fig.write_html(f"../output/visualizations/histogram/{title}.html")
    for compare_col in ["season", "week", "season_type", "venue_indoor", "season_type"]:
        title = f"{col}_histogram_{compare_col}"
        fig = px.histogram(box_scores_df, x=col, title=title, histnorm="percent", color=compare_col)
        fig.write_html(f"../output/visualizations/histogram/{title}.html")
    for y_col in score_cols:
        if y_col != col:
            title = f"{col}_vs_{y_col}_scatter"
            fig = px.scatter(box_scores_df, x=col, y=y_col, title=title)
            fig.write_html(f"../output/visualizations/scatter/{title}.html")


In [91]:
for compare_col in ["season", "week", "season_type", "venue_indoor", "season_type"]:
    for col in ["total_score", "score_differential"]:
        print()
        print(f"{compare_col} by {col}")
        print(box_scores_df.groupby(by=compare_col)[col].describe())
        fig_df = box_scores_df.groupby(by=compare_col)[col].describe().reset_index()
        fig = px.line(fig_df, x=compare_col, y="mean", title=title)
        fig.write_html(f"../output/visualizations/line/{title}.html")


season by total_score
        count       mean        std   min    25%   50%    75%    max
season                                                              
2004     28.0  45.071429  13.770415  19.0  37.00  45.0  53.00   73.0
2005    268.0  41.097015  13.047232  13.0  31.00  41.0  50.00   82.0
2006    268.0  41.578358  14.192807   9.0  31.00  41.0  51.00   90.0
2007    268.0  43.492537  14.611958   3.0  33.00  43.0  53.00   96.0
2008    268.0  44.052239  14.040240  13.0  35.00  43.0  52.00   91.0
2009    268.0  43.279851  13.937261   9.0  34.00  43.0  51.25   96.0
2010    268.0  44.511194  14.387230   9.0  34.75  45.0  53.00   96.0
2011    268.0  44.697761  13.716842   9.0  36.00  43.0  52.00  100.0
2012    268.0  45.966418  14.403066  13.0  36.00  44.0  54.25   97.0
2013    268.0  46.854478  14.440025  19.0  37.00  45.0  55.25   99.0
2014    269.0  45.156134  13.853638   0.0  35.00  44.0  54.00   85.0
2015    268.0  45.578358  13.642486  13.0  36.00  44.0  54.25  101.0
2016    268


season by score_differential
        count      mean        std   min   25%  50%    75%   max
season                                                          
2004     28.0  2.714286  13.077708 -28.0 -5.50  3.0  11.50  30.0
2005    268.0  3.537313  14.711791 -42.0 -6.00  3.0  13.00  49.0
2006    268.0  1.011194  14.289800 -35.0 -8.25  3.0   9.00  41.0
2007    268.0  2.962687  15.239858 -46.0 -7.00  3.0  12.25  45.0
2008    268.0  2.298507  15.172185 -37.0 -7.00  3.0  11.25  44.0
2009    268.0  2.414179  16.477090 -38.0 -7.00  3.0  13.00  59.0
2010    268.0  1.727612  14.919147 -45.0 -6.00  3.0  10.00  42.0
2011    268.0  3.526119  15.180169 -35.0 -7.00  3.0  13.00  55.0
2012    268.0  2.533582  15.525344 -38.0 -7.00  3.0  12.00  58.0
2013    268.0  2.899254  14.193457 -35.0 -4.25  3.0  11.25  43.0
2014    269.0  2.613383  15.535442 -31.0 -8.00  3.0  12.00  52.0
2015    268.0  1.675373  14.229889 -34.0 -7.00  3.0  10.00  40.0
2016    268.0  2.951493  12.931738 -38.0 -5.00  3.0  10.00  

In [79]:
for col in score_cols:
    for compare_col in ["season", "week", "season_type", "venue_indoor", "season_type"]:
        title = f"{col}_scatter_{compare_col}"
        fig = px.scatter(box_scores_df, x=compare_col, y=col, title=title, opacity=0.4)
        fig.write_html(f"../output/visualizations/scatter/{title}.html")

In [68]:
title = f"game_line_percentile"
fig=px.line(box_scores_df.sort_values(by=["score_differential_percentile"]), x="score_differential", y="score_differential_percentile",title=title)
fig.write_html(f"../output/visualizations/line/{title}.html")
for compare_col in ["season", "week", "season_type", "venue_indoor", "season_type"]:
    fig_df = box_scores_df.sort_values(by=[compare_col, "yearly_score_differential_percentile"])
    title = f"game_line_scatter_{compare_col}"
    fig=px.line(fig_df, x="score_differential", y="yearly_score_differential_percentile",title=title, color=compare_col)
    fig.write_html(f"../output/visualizations/line/{title}.html")

In [69]:
title = f"total_line_percentile"
fig=px.line(box_scores_df.sort_values(by=["total_score"]), x="total_score", y="total_score_percentile",title=title)
fig.write_html(f"../output/visualizations/line/{title}.html")
for compare_col in ["season", "week", "season_type", "venue_indoor", "season_type"]:
    fig_df = box_scores_df.sort_values(by=[compare_col, "yearly_total_score_percentile"])
    title = f"total_line_scatter_{compare_col}"
    fig=px.line(fig_df, x="total_score", y="yearly_total_score_percentile",title=title, color=compare_col)
    fig.write_html(f"../output/visualizations/line/{title}.html")